In [10]:
import cv2
import numpy as np

class ImageStitching():
    def __init__(self):
        self.ratio = 0.75  # Adjusted ratio for better matching
        self.min_match = 10
        self.orb = cv2.ORB_create()
        self.smoothing_window_size = 800

    def registration(self, img1, img2):
        # Detect keypoints and descriptors
        kp1, des1 = self.orb.detectAndCompute(img1, None)
        kp2, des2 = self.orb.detectAndCompute(img2, None)
        
        # Match descriptors using BFMatcher with Hamming distance
        matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches = matcher.match(des1, des2)
        matches = sorted(matches, key=lambda x: x.distance)

        # Filter good matches based on the ratio
        good_matches = [m for m in matches if m.distance < self.ratio * matches[-1].distance]

        # Draw matches for visualization
        img3 = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, flags=2)
        cv2.imwrite('matching.jpg', img3)

        # Compute homography if there are enough good matches
        if len(good_matches) > self.min_match:
            image1_kp = np.float32([kp1[m.queryIdx].pt for m in good_matches])
            image2_kp = np.float32([kp2[m.trainIdx].pt for m in good_matches])
            H, status = cv2.findHomography(image2_kp, image1_kp, cv2.RANSAC, 5.0)
            return H
        else:
            raise ValueError("Not enough matches found for homography estimation.")
    
    def create_mask(self, img1, img2, version):
        height_img1, width_img1 = img1.shape[:2]
        width_img2 = img2.shape[1]
        height_panorama = height_img1
        width_panorama = width_img1 + width_img2
        offset = int(self.smoothing_window_size / 2)
        barrier = width_img1 - offset

        mask = np.zeros((height_panorama, width_panorama))
        if version == 'left_image':
            mask[:, barrier - offset:barrier + offset] = np.tile(np.linspace(1, 0, 2 * offset), (height_panorama, 1))
            mask[:, :barrier - offset] = 1
        elif version == 'right_image':
            mask[:, barrier - offset:barrier + offset] = np.tile(np.linspace(0, 1, 2 * offset), (height_panorama, 1))
            mask[:, barrier + offset:] = 1
        return cv2.merge([mask, mask, mask])
    
    def blending(self, img1, img2):
        # Perform image registration to find homography
        H = self.registration(img1, img2)

        # Image dimensions
        height_img1, width_img1 = img1.shape[:2]
        width_img2 = img2.shape[1]
        height_panorama = height_img1
        width_panorama = width_img1 + width_img2

        # Prepare the panorama with initial image and mask
        panorama1 = np.zeros((height_panorama, width_panorama, 3))
        mask1 = self.create_mask(img1, img2, version='left_image')
        panorama1[0:height_img1, 0:width_img1] = img1
        panorama1 *= mask1

        # Warp second image and apply the mask
        mask2 = self.create_mask(img1, img2, version='right_image')
        panorama2 = cv2.warpPerspective(img2, H, (width_panorama, height_panorama)) * mask2
        
        # Combine the two panoramas
        result = panorama1 + panorama2

        # Crop the final result to remove black borders
        rows, cols = np.where(result[:, :, 0] != 0)
        min_row, max_row = min(rows), max(rows) + 1
        min_col, max_col = min(cols), max(cols) + 1
        final_result = result[min_row:max_row, min_col:max_col]
        return final_result





In [11]:

argv1 = '../data/undistorted_image.png'
argv2 = '../data/undistorted_image2.png'

img1 = cv2.imread(argv1)
img2 = cv2.imread(argv2)
final=Image_Stitching().blending(img1,img2)
cv2.imwrite('panorama.jpg', final)

UnboundLocalError: local variable 'H' referenced before assignment

In [44]:
def correct_distortion(img, dist_coeffs= [-1, 1, 0, 0]):
    """
    校正图像畸变的函数。

    参数：
    img (numpy.ndarray): 输入图像
    K (numpy.ndarray): 相机内参矩阵
    dist_coeffs (numpy.ndarray): 畸变系数

    返回：
    numpy.ndarray: 校正后的图像
    """
    # 假设相机内参矩阵（需要根据具体相机参数调整）
    K = np.array([[800, 0, img.shape[1] / 2],
                  [0, 800, img.shape[0] / 2],
                  [0, 0, 1]], dtype=np.float32)

    # 定义畸变系数
    dist_coeffs = np.array(dist_coeffs)
    # 计算新的相机矩阵
    new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(K, dist_coeffs, (img.shape[1], img.shape[0]), 1, (img.shape[1], img.shape[0]))

    # 畸变校正
    corrected_img = cv2.undistort(img, K, dist_coeffs, None, new_camera_matrix)

    # 裁剪图像
    x, y, w, h = roi
    corrected_img = corrected_img[y:y+h, x:x+w]

    return corrected_img

In [45]:
im11 = correct_distortion(img1)
cv2.imwrite("../data/video1_frame1_d.jpg", im11)

True

In [28]:
cv2.imwrite('../data/video1_frame1_d.jpg', correct_distortion(img1))
cv2.imwrite('../data/video2_frame1_d.jpg', correct_distortion(img2))


True

In [4]:



import cv2
import numpy as np

def perspective_correction(image_path, output_path, points):
    # 读取图像
    img = cv2.imread(image_path)
    
    # 获取图像尺寸
    h, w = img.shape[:2]

    # 指定输入图像的四个角点
    pts1 = np.float32(points)

    # 计算透视变换矩阵
    matrix = cv2.getPerspectiveTransform(pts1, np.float32([[0, 0], [1, 0], [1, 1], [0, 1]]))

    # 应用透视变换来找到新的边界框
    transformed_points = cv2.perspectiveTransform(np.array([points], dtype=np.float32), matrix)
    transformed_points = transformed_points[0]

    # 计算新图像的边界
    min_x = np.min(transformed_points[:, 0])
    max_x = np.max(transformed_points[:, 0])
    min_y = np.min(transformed_points[:, 1])
    max_y = np.max(transformed_points[:, 1])

    # 新图像的尺寸
    new_w = int(max_x - min_x)
    new_h = int(max_y - min_y)

    # 调整输出的四个角点
    pts2 = np.float32([
        [-min_x, -min_y],
        [new_w - min_x, -min_y],
        [new_w - min_x, new_h - min_y],
        [-min_x, new_h - min_y]
    ])

    # 重新计算透视变换矩阵
    matrix = cv2.getPerspectiveTransform(pts1, pts2)

    # 进行透视变换
    result = cv2.warpPerspective(img, matrix, (new_w, new_h))

    # 保存输出图像
    cv2.imwrite(output_path, result)

# 使用示例
# 输入图像四个角点的坐标 (根据实际情况调整)
points = [[100, 100], [500, 100], [500, 500], [100, 500]]
perspective_correction('../data/video1_frame1.jpg', 'output.jpg', points)

